### Notes for Harry 
#### by August George, 2022

ref on implementation: https://si.sciml.ai/dev/ 

ref on transporter modeling: https://journals.plos.org/ploscompbiol/article/authors?id=10.1371/journal.pcbi.1007884

In [7]:
using Catalyst, DifferentialEquations, StructuralIdentifiability, ModelingToolkit

In [8]:
### define reaction network in Catalyst
# notes: 
# H = H+/ion, S = substrate
# 'OF' = outward facing conformation, 'IF' = inward facing conformation
# 'H_out' and 'S_out' = fixed external concentrations of the ion and substrate, respectively
# 'H_in' and 'S_in' = internal concentrations of the ion and substrate, respectively
# 'Hb' and 'Sb' are the ion and/or substrate bound to the protein, respectively

# single antiporter pathway (ion and substrate transported in opposite directions)
rn = @reaction_network begin
  (k1_f*H_out, k1_r), OF <--> OF_Hb
  (k2_f, k2_r), OF_Hb <--> IF_Hb
  (k3_f, k3_r), IF_Hb + S_in <--> IF_Hb_Sb
  (k4_f, k4_r), IF_Hb_Sb <--> IF_Sb + H_in
  (k5_f, k5_r), IF_Sb <--> OF_Sb
  (k6_f, k6_r*S_out), OF_Sb <--> OF
end H_out S_out k1_f k1_r k2_f k2_r k3_f k3_r k4_f k4_r k5_f k5_r k6_f k6_r

Model ##ReactionSystem#301 with 12 equations
States (8):
  OF(t)
  OF_Hb(t)
  IF_Hb(t)
  S_in(t)
  IF_Hb_Sb(t)
  IF_Sb(t)
⋮
Parameters (14):
  H_out
  S_out
  k1_f
  k1_r
  k2_f
  k2_r
⋮

In [9]:
### latexify
odesys = convert(ODESystem, rn)
equations(odesys)

8-element Vector{Equation}:
 Differential(t)(OF(t)) ~ k1_r*OF_Hb(t) + k6_f*OF_Sb(t) - H_out*k1_f*OF(t) - S_out*k6_r*OF(t)
 Differential(t)(OF_Hb(t)) ~ k2_r*IF_Hb(t) + H_out*k1_f*OF(t) - k1_r*OF_Hb(t) - k2_f*OF_Hb(t)
 Differential(t)(IF_Hb(t)) ~ k3_r*IF_Hb_Sb(t) + k2_f*OF_Hb(t) - k2_r*IF_Hb(t) - k3_f*IF_Hb(t)*S_in(t)
 Differential(t)(S_in(t)) ~ k3_r*IF_Hb_Sb(t) - k3_f*IF_Hb(t)*S_in(t)
 Differential(t)(IF_Hb_Sb(t)) ~ k3_f*IF_Hb(t)*S_in(t) + k4_r*H_in(t)*IF_Sb(t) - k3_r*IF_Hb_Sb(t) - k4_f*IF_Hb_Sb(t)
 Differential(t)(IF_Sb(t)) ~ k4_f*IF_Hb_Sb(t) + k5_r*OF_Sb(t) - k5_f*IF_Sb(t) - k4_r*H_in(t)*IF_Sb(t)
 Differential(t)(H_in(t)) ~ k4_f*IF_Hb_Sb(t) - k4_r*H_in(t)*IF_Sb(t)
 Differential(t)(OF_Sb(t)) ~ k5_f*IF_Sb(t) + S_out*k6_r*OF(t) - k5_r*OF_Sb(t) - k6_f*OF_Sb(t)

In [11]:
# make ODE model w/ 3 obersved quantities (H_out, S_out, and dH_in/dt)
rn_ode = @ODEmodel(
    OF'(t) = k1_r*OF_Hb(t)+k6_f*OF_Sb(t)−H_out*k1_f*OF(t)−S_out*k6_r*OF(t),
    OF_Hb'(t) = k2_r*IF_Hb(t)+H_out*k1_f*OF(t)−k1_r*OF_Hb(t)−k2_f*OF_Hb(t),
    IF_Hb'(t) = k3_r*IF_Hb_Sb(t)+k2_f*OF_Hb(t)−k2_r*IF_Hb(t)−k3_f*IF_Hb(t)*S_in(t),
    S_in'(t) = k3_r*IF_Hb_Sb(t)−k3_f*IF_Hb(t)*S_in(t),
    IF_Hb_Sb'(t) = −k3_r*IF_Hb_Sb(t)−k4_f*IF_Hb_Sb(t)+k3_f*IF_Hb(t)*S_in(t)+k4_r*H_in(t)*IF_Sb(t),
    IF_Sb'(t) = k4_f*IF_Hb_Sb(t)+k5_r*OF_Sb(t)−k5_f*IF_Sb(t)−k4_r*H_in(t)*IF_Sb(t),
    H_in'(t) = k4_f*IF_Hb_Sb(t)−k4_r*H_in(t)*IF_Sb(t),
    OF_Sb'(t) = k5_f*IF_Sb(t)+S_out*k6_r*OF(t)−k5_r*OF_Sb(t)−k6_f*OF_Sb(t), 
    
    y1(t) = H_out, 
    y2(t) = S_out, 
    y3(t) = H_in'(t)
)

# run local identifiability w/ 99% probability
@time local_id = assess_local_identifiability(rn_ode, 0.99)
println(local_id)

# # run global identifiability w/ 99% probability (might be really slow)
# @time global_id = assess_global_identifiability(rn_ode, 0.99)
# println(global_id)

  0.402104 seconds (2.85 M allocations: 322.915 MiB, 26.95% gc time, 0.96% compilation time)
Dict{Nemo.fmpq_mpoly, Bool}(IF_Sb => 1, S_in => 1, S_out => 1, k3_f => 1, k1_f => 1, k6_f => 1, k2_f => 1, k4_f => 1, H_out => 1, k1_r => 1, OF_Hb => 1, IF_Hb => 1, k5_f => 1, k3_r => 1, k2_r => 1, H_in => 1, OF_Sb => 1, OF => 1, k6_r => 1, k4_r => 1, k5_r => 1, IF_Hb_Sb => 1)


┌ Info: Summary of the model:
└ @ StructuralIdentifiability /Users/georgeau/.julia/packages/StructuralIdentifiability/EQH60/src/ODE.jl:303
┌ Info: State variables: S_in, IF_Hb_Sb, OF, H_in, IF_Hb, OF_Sb, IF_Sb, OF_Hb
└ @ StructuralIdentifiability /Users/georgeau/.julia/packages/StructuralIdentifiability/EQH60/src/ODE.jl:304
┌ Info: Parameters: k2_f, k5_f, k4_r, S_out, k3_r, k3_f, k4_f, k5_r, k1_f, H_out, k6_f, k1_r, k6_r, k2_r
└ @ StructuralIdentifiability /Users/georgeau/.julia/packages/StructuralIdentifiability/EQH60/src/ODE.jl:305
┌ Info: Inputs: 
└ @ StructuralIdentifiability /Users/georgeau/.julia/packages/StructuralIdentifiability/EQH60/src/ODE.jl:306
┌ Info: Outputs: y3, y1, y2
└ @ StructuralIdentifiability /Users/georgeau/.julia/packages/StructuralIdentifiability/EQH60/src/ODE.jl:307
